<a href="https://colab.research.google.com/github/dbahiense/MachineLearning/blob/main/03Boston(Code).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **III. PROBLEMA DE REGRESSÃO**

# 1. Preparativos
---

## 1.1 Importar Bibliotecas e Módulos

In [28]:
#importar bibliotecas e módulos necessários
import os

import numpy
from numpy import arange
from numpy import set_printoptions

from matplotlib import pyplot

from pandas import read_csv
from pandas import set_option
from pandas.plotting import scatter_matrix

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error

## 1.2 Parâmetros

In [2]:
# Opções e métricas para treino e teste
test_size = 0.20
seed = 7
num_folds = 10
scoring = 'neg_mean_squared_error'

---
**Notas:**  
- `test_size`é um float...

- `seed` é um inteiro utilizado em `random_state` que, por sua vez, indica o grau de *embaralhamento* (shuffling) aplicado aos dados antes do fatiamento.  
https://scikit-learn.org/stable/glossary.html#term-random_state  
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

# 2. Dados
---

## 2.1 Obter Dados

In [3]:
#endereço dos dados originais
#pode ser modificado para endereço local na própria máquina
url = "https://gist.githubusercontent.com/dbahiense/10f30ddee5ea0e44b63bb08db1010c9d/raw/368fb94696d70253dc9cd4683bec6fad4fcf22ee/housing.csv"

#nome dos atributos
columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

#lê o arquivo
#delimitador é espaço em branco (delim_whitespace = True)
data = read_csv(url, delim_whitespace=True, names=columns)

## 2.2 Entender Dados

### 2.2.1 Descrever Dados

In [ ]:
#mostra as dimensões do dataset
shape = data.shape #(linhas, colunas)
shape

In [ ]:
#mostra as primeiras n instâncias dos dados
peek = data.head(10) #head(n) para mostrar as primeiras n linhas, default n = 5
peek

In [ ]:
#mostra os tipos de dados de cada atributo (coluna)
types = data.dtypes
types

### 2.2.2 Estatística Descritiva  

**mais:** https://www.investopedia.com/terms/d/descriptive_statistics.asp

In [ ]:
#mostra os dados estatísticos de cada atributo (coluna)
set_option('precision', 1) #precisão de uma casa decimal após a vírgula
description = data.describe()
description

In [ ]:
#mostra a correlação entre os atributos
set_option('precision', 2) #precisão de duas casas decimais após a vírgula
#métodos podem ser: pearson, spearman ou kendall
correlations = data.corr(method = 'pearson')
correlations

In [ ]:
#mostra a distorção (obliquidade) das distribuições univariadas
skew = data.skew(numeric_only=True)
skew

## 2.3 Visualizar Dados

### 2.3.1 Gráficos Univariados

#### 2.3.1.1 Histograma

In [ ]:
data.hist(sharex=False, sharey=False, xlabelsize=1, ylabelsize=1)
pyplot.show()

#### 2.3.1.2 Gráficos de Densidade

In [ ]:
#gráficos de densidade
#atenção ao layout para caberem todos os gráficos (x * y >= n° de atributos)
data.plot(kind='density', subplots=True, layout=(4,4), sharex=False, sharey=False, legend=False, fontsize=2)
pyplot.show()

#### 2.3.1.3 Box Plot

In [ ]:
#box plot
#atenção ao layout para caberem todos os gráficos (x * y >= n° de atributos)
data.plot(kind='box', subplots=True, layout=(4,4), sharex=False, sharey=False, fontsize=8)
pyplot.show()

### 2.3.2 Gráficos Multivariados

#### 2.3.2.1 Matriz de Correlação

In [ ]:
#matriz de correlação
#corr = 1 atributos diretamente correlacionados
#corr = 0 atributos não relacionados
#corr = -1 atributos inversamente correlacionados
fig = pyplot.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(correlations, vmin=-1, vmax=1, interpolation='none')
fig.colorbar(cax)
ticks = numpy.arange(0,14,1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(columns)
ax.set_yticklabels(columns)
pyplot.show()

#### 2.3.2.2 Scatter Plot

In [ ]:
scatter_matrix(data)
pyplot.show()

## 2.4 Preparar Dados

O conjunto de dados não apresenta valores faltantes nem redundantes. Entretanto, os atributos estão em diversas escalas. Para corrigir isso iremos padronizar os dados.

### 2.4.1 Padronizar Dados

In [ ]:
#dispõe os dados em forma de array
array = data.values

#fatia os dados em um conjunto de inputs X e outro de output y
X = array[ : , 0:13]
y = array[ : , 13]

#padroniza os dados (fit and transform)
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

#imprime os dados padronizados
set_printoptions(precision=3) #precisão de três casas decimais após a vírgula
print(X[ : , 0:13])

---  
**Notas:**  
`X = array[ : , 0:13]` trata-se de um fatiamento (slice) no formato `[start, stop]`.  
Nesse caso, `start = : ` significa `start` da primeira à última linha e  
`stop = 0:13` significa `stop`da primeira à décima terceira coluna, ou melhor dizendo, da coluna de índice 0 à coluna de índice 13.  
Não podemos nos esquecer que no caso do Python os intervalos geralmente são [fechados, abertos[.  

https://docs.python.org/3/tutorial/introduction.html#lists  

`StandardScaler()`

### 2.4.2 Reduzir Dados

### 2.4.3 Fatiar Dados
Separa um conjunto de dados para o treinamento e um outro conjunto de dados para os testes.

In [16]:
#From documentation: split array or matrix into random train and test subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

---
**Nota:**  
`train_test_split`: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

#3. Algoritmos
---

## 3.1 Avaliar Algoritmos

### 3.1.1 Selecionar Algoritmos

In [17]:
#seleção de algoritmos
models = []
models.append(('LR', LinearRegression()))
models.append(('LASSO', Lasso()))
models.append(('EN', ElasticNet()))
models.append(('KNN', KNeighborsRegressor()))
models.append(('CART', DecisionTreeRegressor()))
models.append(('SVR', SVR()))

---
**Nota:**  
Por que selecionamos esses algoritmos?

### 3.1.2 Comparar Algoritmos

In [ ]:
#lista dos nomes dos algoritmos
names = []
#lista dos resultados dos algoritmos
results = []

print(f" {'Algoritmo'} | {'Média':<8} | {'Desvio'}")
print(32 * '-')

for name, model in models:
  kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
  cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)

  #imprimir nome do algoritmo, média e desvio padrão
  msg = f" {name:<9} | {cv_results.mean():.4f} | {cv_results.std():.4f}"
  print(msg)
  print(32 * '-')

  #insere nome do algoritmo na lista names 
  names.append(name)
  #insere resultado do algoritmo na lista results
  results.append(cv_results)

### 3.1.3 Visualizar Comparação

In [ ]:
#comparação gráfica dos algoritmos
fig = pyplot.figure()
fig.suptitle('Comparação de Algoritmos')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

## 3.2 Melhorar Desempenho do Melhor Algoritmo

### 3.2.1 Ajustar Parâmetros

In [20]:
k_values = numpy.array([1,3,5,7,9,11,13,15,17,19,21,23,24,25,27,29])

#modelo que apresentou o melhor resultado
model = KNeighborsRegressor()

#ordena os parâmetros a serem testados
param_grid = dict(n_neighbors=k_values)
kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)

grid_result = grid.fit(X_train, y_train)

### 3.2.2 Apresentar Resultados

In [ ]:
#resultado dos testes com os diferentes parâmetros (n_neighbors / k_values)
print("Melhor resultado: %f usando %s \n" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
  print("%f (%f) >>> %r" % (mean, stdev, param))

## 3.3 Melhorar Desempenho com Conjunto de Algoritmos

### 3.3.1 Selecionar Conjunto de Algoritmos (Ensembles)

In [22]:
#conjuntos (ensembles)
ensembles = []

#boosting
ensembles.append(('ScaledAB', Pipeline([('Scaler', StandardScaler()),('AB', AdaBoostRegressor())])))
ensembles.append(('ScaledGBM', Pipeline([('Scaler', StandardScaler()),('GBM', GradientBoostingRegressor())])))

#bagging
ensembles.append(('ScaledRF', Pipeline([('Scaler', StandardScaler()),('RF', RandomForestRegressor(n_estimators=10))])))
ensembles.append(('ScaledET', Pipeline([('Scaler', StandardScaler()),('ET', ExtraTreesRegressor(n_estimators=10))])))

---
**Nota:**  
Por que selecionamos esses conjuntos de algoritmos (ensembles)?

### 3.3.2 Comparar Conjuntos de Algoritmos

In [ ]:
#lista dos nomes dos algoritmos
names = []
#lista dos resultados dos algoritmos
results = []

for name, model in ensembles:
  kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
  cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)

  #média, desvio padrão, algoritmo
  msg = f'{cv_results.mean():.4f} ({cv_results.std():.4f}): {name}'
  print(msg)

  #insere nome do algoritmo na lista names
  names.append(name)
  #insere resultado do algoritmo na lista results
  results.append(cv_results)

### 3.3.3 Visualizar Comparação

In [ ]:
fig = pyplot.figure()
fig.suptitle('Comparação dos Conjuntos de Algoritmos')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

## 3.4 Melhorar Desempenho do Melhor Conjunto de Algoritmos

### 3.4.1 Ajustar Parâmetros


In [25]:
#refinar GBM

param_grid = dict(n_estimators=numpy.array([50,100,300,450,500,600,650,700]))

model = GradientBoostingRegressor(random_state=seed)

kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)

grid_result = grid.fit(X_train, y_train)

### 3.4.2 Apresentar Resultados

In [ ]:
#resultado
print("Melhor resultado: %f usando %s \n" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
  print("%f (%f) >>> %r" % (mean, stdev, param))

##4. Finalizar Modelo

## 4.1 Gravar Modelo

In [ ]:
model = GradientBoostingRegressor(random_state=seed, n_estimators=650)
model.fit(X_train, y_train)

#gravar o modelo no disco
#filename = 'boston_model.sav'
#dump(model, filename)

## 4.2 Carregar Modelo

In [31]:
#carregar modelo do disco 
#model = load(filename)

## 4.3 Usar Modelo

In [ ]:
prediction = model.predict(X_test)
print(mean_squared_error(y_test, prediction))